In [68]:
import pandas as pd
pd.set_option('display.max_columns', None) 
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [69]:
input_path = 'H:\RediMinds\VCQI'
df_filter2_codes = pd.read_csv(input_path+"\VCQI_clean.csv")

In [70]:
# sepearte target variable from predictors for furthur processing
X = df_filter2_codes.drop(columns=['INTRA-OP COMPLICATION'])
Y = df_filter2_codes['INTRA-OP COMPLICATION']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2852 entries, 0 to 2851
Data columns (total 33 columns):
PROC NAME                          2852 non-null int64
GENDER                             2852 non-null int64
AGE AT SURGERY                     2852 non-null float64
MARITAL STATUS                     2852 non-null int64
RACE                               2852 non-null int64
WEIGHT                             2852 non-null float64
HEIGHT                             2852 non-null float64
BMI                                2852 non-null float64
CLINICAL SIZE (mm)                 2852 non-null float64
SYMPTOMS                           2852 non-null int64
SOLITARY KIDNEY                    2852 non-null int64
BILATERALITY OF TUMOR              2852 non-null int64
SIDE OF TUMOR                      2852 non-null int64
SIDE OF SURGERY                    2852 non-null int64
FACE                               2852 non-null int64
TUMOR lOCATION                     2852 non-null int64
PRE

In [71]:
X.select_dtypes('float').columns

Index(['AGE AT SURGERY', 'WEIGHT', 'HEIGHT', 'BMI', 'CLINICAL SIZE (mm)',
       'PRE-OP HB', 'PRE-OP HT', 'PRE-OP WBC', 'PRE-OP CREAT', 'PRE-OP EGFR',
       'NO OF LESIONS'],
      dtype='object')

In [72]:
# Create dummy variables
with open (input_path+'\cat_col', 'rb') as fp:
    cat_col = pickle.load(fp)


nn_data = X.copy()
for col in cat_col:
    if not col == 'INTRA-OP COMPLICATION':
        nn_data = nn_data.merge(pd.get_dummies(X[col], prefix=col), how = 'left', left_index = True, right_index =True) # create dummy variables
        nn_data = nn_data.drop(labels = col, axis = 'columns') # drop original variables

In [73]:
nn_data.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,50.0,57.00,168.0,20.20,7.0,13.2,50.0,7800.0,0.63,100.030000,1.0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
1,69.0,68.60,161.3,26.37,74.0,14.7,44.2,6400.0,0.80,71.119672,1.0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1
2,83.0,83.65,164.0,31.10,80.0,12.5,38.9,5590.0,0.97,54.844682,1.0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0
3,66.0,107.00,180.0,32.78,73.0,17.0,50.8,8000.0,0.93,81.290783,1.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0
4,61.0,82.90,171.0,28.35,8.0,12.1,37.1,7710.0,1.10,68.053260,1.0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0


In [74]:
# Split data in test and train
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(nn_data, Y, test_size=0.30, random_state=42, stratify = Y)

from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(nn_data, Y, test_size=0.20, random_state=42, stratify = Y)

In [75]:
print("Number of records in trainset {}".format(len(x_train)))
print("Number records in testset {}".format(len(x_test)))

Number of records in trainset 2281
Number records in testset 856


In [76]:
sum(y_train)/len(y_train)

0.06444541867601929

In [77]:
sum(y_test)/len(y_test)

0.06425233644859812

In [78]:
x_train.describe()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
count,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.0,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.0,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000,2281.000000
mean,58.307146,82.886502,169.659452,29.094038,32.689031,13.749943,40.839259,7620.468601,0.979511,80.256758,1.063218,1.0,0.352915,0.643577,0.003507,0.025866,0.504603,0.367383,0.077598,0.024551,0.028935,0.088558,0.422183,0.058308,0.402017,0.575625,0.172293,0.235423,0.016659,0.127576,0.841736,0.030688,0.402017,0.568610,0.029373,0.020605,0.305129,0.369136,0.305129,0.000438,0.494082,0.003069,0.502411,0.424375,0.226217,0.349408,0.264358,0.331434,0.135905,0.268303,0.199474,0.339763,0.460763,0.378781,0.000438,0.430075,0.090311,0.100395,0.291977,0.365629,0.091626,0.250767,0.220079,0.609382,0.022359,0.148181,0.100833,0.639193,0.226217,0.023235,0.004384,0.006138,0.0,0.004822,0.525208,0.203858,0.266111,0.017098,0.601491,0.201228,0.180184,0.231039,0.640070,0.031565,0.097326,0.358176,0.348531,0.096449,0.196843,0.328803,0.217448,0.145550,0.308198,0.029373,0.567733,0.283209,0.119684
std,12.415875,16.014082,9.399481,5.569943,16.289685,1.596923,4.033790,2289.636811,0.376386,21.836862,0.264819,0.0,0.477981,0.479047,0.059131,0.158770,0.500088,0.482198,0.267596,0.154785,0.167660,0.284166,0.494016,0.234376,0.490413,0.494356,0.377718,0.424356,0.128020,0.333690,0.365068,0.172510,0.490413,0.495379,0.168887,0.142089,0.460563,0.482677,0.460563,0.020938,0.500075,0.055324,0.500104,0.494356,0.418473,0.476888,0.441087,0.470832,0.342763,0.44

In [79]:
# Standardize the numeric columns
from sklearn.preprocessing import StandardScaler
stdc = StandardScaler()
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
x_valid.reset_index(drop=True, inplace=True)
y_valid.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
float_col = X.select_dtypes('float').columns
x_train[float_col] = pd.DataFrame(stdc.fit_transform(x_train[float_col]),columns=float_col).copy()
x_valid[float_col] = pd.DataFrame(stdc.fit_transform(x_valid[float_col]),columns=float_col).copy()
x_test[float_col] = pd.DataFrame(stdc.transform(x_test[float_col]),columns=float_col).copy()

H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [81]:
sm = SMOTE(random_state=2)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [82]:
x_train = pd.DataFrame(x_train, columns=x_test.columns).copy()
y_train = pd.Series(y_train)
x_train.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,-0.105303,0.194465,-0.070174,0.162688,0.448908,0.782962,0.052255,-0.445788,0.506210,-0.698823,-0.238774,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.297495,-0.587518,0.866253,-1.051236,-0.410720,-1.033428,-1.299127,0.008890,-0.477039,0.011600,-0.049924,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.297495,-0.087848,-0.032929,-0.072555,-0.472122,-0.219184,0.052255,0.296851,-0.795931,0.354673,-0.238774,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1.908689,-1.304544,-2.304830,-0.140793,0.141898,-0.594989,-0.555247,0.410431,-1.247694,1.612092,-0.238774,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.539174,-0.087848,-0.032929,2.138006,-0.963338,0.012563,0.052255,0.008890,-0.131573,0.058921,-0.238774,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [85]:
# input layer
input_layer = keras.layers.Input(shape=(x_train.shape[1],), name = "input_layer")
x = keras.layers.Dense(128, name = 'Dense_1',activation='relu')(input_layer)
x = keras.layers.Dropout(0.5, name=  'Dropout_1')(x)
x = keras.layers.Dense(128, name = 'Dense_2',activation='relu')(x)
x = keras.layers.Dropout(0.5, name=  'Dropout_2')(x)
main_output = keras.layers.Dense(1, activation='sigmoid',name='main_output')(x)

model = keras.Model(inputs= input_layer, outputs=main_output)

# compiling the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve = 'ROC',name = 'AUC_ROC'),
                       tf.keras.metrics.AUC(curve = 'PR', name = 'AUC_PR')],
              )

# Keras callback. The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',restore_best_weights=True)


model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 97)]              0         
_________________________________________________________________
Dense_1 (Dense)              (None, 128)               12544     
_________________________________________________________________
Dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
Dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
Dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
main_output (Dense)          (None, 1)                 129       
Total params: 29,185
Trainable params: 29,185
Non-trainable params: 0
_______________________________________________________

In [86]:
model.fit(x = x_train, y = y_train,
          validation_data = (x_valid, y_valid),
          epochs = 50,
          callbacks = [early_stop]
     )


Train on 4268 samples, validate on 571 samples
Epoch 1/50
4268/4268 [==============================] - ETA: 24s - loss: 0.7688 - AUC_ROC: 0.4395 - AUC_PR: 0.524 - ETA: 1s - loss: 0.7269 - AUC_ROC: 0.5238 - AUC_PR: 0.515 - ETA: 0s - loss: 0.7228 - AUC_ROC: 0.5375 - AUC_PR: 0.52 - ETA: 0s - loss: 0.7178 - AUC_ROC: 0.5459 - AUC_PR: 0.53 - ETA: 0s - loss: 0.7145 - AUC_ROC: 0.5512 - AUC_PR: 0.53 - ETA: 0s - loss: 0.7094 - AUC_ROC: 0.5577 - AUC_PR: 0.54 - 1s 320us/sample - loss: 0.7065 - AUC_ROC: 0.5619 - AUC_PR: 0.5473 - val_loss: 0.6932 - val_AUC_ROC: 0.7573 - val_AUC_PR: 0.1433
Epoch 2/50
4268/4268 [==============================] - ETA: 0s - loss: 0.6431 - AUC_ROC: 0.6409 - AUC_PR: 0.75 - ETA: 0s - loss: 0.6837 - AUC_ROC: 0.5990 - AUC_PR: 0.59 - ETA: 0s - loss: 0.6748 - AUC_ROC: 0.6125 - AUC_PR: 0.59 - ETA: 0s - loss: 0.6722 - AUC_ROC: 0.6231 - AUC_PR: 0.60 - ETA: 0s - loss: 0.6670 - AUC_ROC: 0.6341 - AUC_PR: 0.61 - ETA: 0s - loss: 0.6633 - AUC_ROC: 0.6398 - AUC_PR: 0.62 - 0s 75us/sample

4268/4268 [==============================] - ETA: 0s - loss: 0.3435 - AUC_ROC: 0.9292 - AUC_PR: 0.91 - ETA: 0s - loss: 0.3356 - AUC_ROC: 0.9361 - AUC_PR: 0.93 - ETA: 0s - loss: 0.3651 - AUC_ROC: 0.9180 - AUC_PR: 0.91 - ETA: 0s - loss: 0.3600 - AUC_ROC: 0.9209 - AUC_PR: 0.91 - ETA: 0s - loss: 0.3595 - AUC_ROC: 0.9213 - AUC_PR: 0.91 - ETA: 0s - loss: 0.3528 - AUC_ROC: 0.9247 - AUC_PR: 0.92 - 0s 69us/sample - loss: 0.3530 - AUC_ROC: 0.9246 - AUC_PR: 0.9205 - val_loss: 0.3475 - val_AUC_ROC: 0.8860 - val_AUC_PR: 0.5211
Epoch 17/50
4268/4268 [==============================] - ETA: 0s - loss: 0.2540 - AUC_ROC: 0.9784 - AUC_PR: 0.96 - ETA: 0s - loss: 0.3389 - AUC_ROC: 0.9318 - AUC_PR: 0.92 - ETA: 0s - loss: 0.3479 - AUC_ROC: 0.9279 - AUC_PR: 0.92 - ETA: 0s - loss: 0.3451 - AUC_ROC: 0.9286 - AUC_PR: 0.92 - ETA: 0s - loss: 0.3407 - AUC_ROC: 0.9308 - AUC_PR: 0.92 - 0s 63us/sample - loss: 0.3392 - AUC_ROC: 0.9312 - AUC_PR: 0.9262 - val_loss: 0.3350 - val_AUC_ROC: 0.8872 - val_AUC_PR: 0.5222
Epoch 

Epoch 32/50
4268/4268 [==============================] - ETA: 0s - loss: 0.3188 - AUC_ROC: 0.9683 - AUC_PR: 0.96 - ETA: 0s - loss: 0.1992 - AUC_ROC: 0.9794 - AUC_PR: 0.97 - ETA: 0s - loss: 0.2138 - AUC_ROC: 0.9734 - AUC_PR: 0.96 - ETA: 0s - loss: 0.2224 - AUC_ROC: 0.9705 - AUC_PR: 0.96 - ETA: 0s - loss: 0.2262 - AUC_ROC: 0.9697 - AUC_PR: 0.96 - ETA: 0s - loss: 0.2233 - AUC_ROC: 0.9706 - AUC_PR: 0.96 - 0s 69us/sample - loss: 0.2249 - AUC_ROC: 0.9700 - AUC_PR: 0.9635 - val_loss: 0.2801 - val_AUC_ROC: 0.8780 - val_AUC_PR: 0.5227
Epoch 33/50
4268/4268 [==============================] - ETA: 0s - loss: 0.2541 - AUC_ROC: 0.9663 - AUC_PR: 0.95 - ETA: 0s - loss: 0.2244 - AUC_ROC: 0.9711 - AUC_PR: 0.96 - ETA: 0s - loss: 0.2164 - AUC_ROC: 0.9738 - AUC_PR: 0.97 - ETA: 0s - loss: 0.2234 - AUC_ROC: 0.9715 - AUC_PR: 0.96 - ETA: 0s - loss: 0.2169 - AUC_ROC: 0.9733 - AUC_PR: 0.97 - 0s 65us/sample - loss: 0.2145 - AUC_ROC: 0.9738 - AUC_PR: 0.9702 - val_loss: 0.2711 - val_AUC_ROC: 0.8776 - val_AUC_PR: 0

Epoch 48/50
4268/4268 [==============================] - ETA: 0s - loss: 0.1911 - AUC_ROC: 0.9757 - AUC_PR: 0.96 - ETA: 0s - loss: 0.1662 - AUC_ROC: 0.9816 - AUC_PR: 0.96 - ETA: 0s - loss: 0.1599 - AUC_ROC: 0.9837 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1554 - AUC_ROC: 0.9848 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1546 - AUC_ROC: 0.9848 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1530 - AUC_ROC: 0.9853 - AUC_PR: 0.98 - 0s 70us/sample - loss: 0.1525 - AUC_ROC: 0.9854 - AUC_PR: 0.9812 - val_loss: 0.2482 - val_AUC_ROC: 0.8640 - val_AUC_PR: 0.5249
Epoch 49/50
4268/4268 [==============================] - ETA: 0s - loss: 0.1657 - AUC_ROC: 0.9879 - AUC_PR: 0.99 - ETA: 0s - loss: 0.1392 - AUC_ROC: 0.9868 - AUC_PR: 0.98 - ETA: 0s - loss: 0.1391 - AUC_ROC: 0.9879 - AUC_PR: 0.98 - ETA: 0s - loss: 0.1374 - AUC_ROC: 0.9883 - AUC_PR: 0.98 - ETA: 0s - loss: 0.1418 - AUC_ROC: 0.9872 - AUC_PR: 0.98 - ETA: 0s - loss: 0.1405 - AUC_ROC: 0.9875 - AUC_PR: 0.98 - 0s 70us/sample - loss: 0.1393 - AUC_ROC: 0.9877 - AUC_PR: 0.984

In [87]:
# predict on test set
pd.DataFrame(model.predict(x_test))

,0
0,3.308058e-06
1,0.000000e+00
2,3.995794e-01
3,6.848574e-05
4,8.262315e-01
5,2.422929e-04
6,1.738966e-04
7,7.361174e-06
8,1.743740e-02
9,1.361156e-01


In [88]:
results_nn = pd.DataFrame(model.predict(x_test), columns=['pred_prob'])
results_nn['pred_label'] =  results_nn['pred_prob'].apply(lambda x: 1 if x>=0.5 else 0)
results_nn['true_label'] = np.array(y_test)

In [89]:
# NeuralNetwork Score Raw Data
print("\n Model Balanced Accuracy: \n" + str(metrics.balanced_accuracy_score(results_nn['true_label'], results_nn['pred_label'])))
print("\n Confusion Matrix : \n"+str(metrics.confusion_matrix(results_nn['true_label'], results_nn['pred_label'])))
print("\n Classification Report: \n"+ str(metrics.classification_report(results_nn['true_label'], results_nn['pred_label'])))
print("\n AUC-ROC: \n"+ str(metrics.roc_auc_score(results_nn['true_label'], results_nn['pred_prob'])))


def calc_aucpr_data(result):
    y_ACTUAL = result['true_label']
    scores_prob = result['pred_prob']
    yhat = result['pred_label']
    precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
    prc_auc = metrics.auc(recall,precision)
    return prc_auc

print("\n AUC-ROC: \n"+ str(calc_aucpr_data(results_nn)))


 Model Balanced Accuracy: 
0.8039042106457837

 Confusion Matrix : 
[[749  52]
 [ 18  37]]

 Classification Report: 
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       801
           1       0.42      0.67      0.51        55

    accuracy                           0.92       856
   macro avg       0.70      0.80      0.73       856
weighted avg       0.94      0.92      0.93       856


 AUC-ROC: 
0.9170582226762002

 AUC-ROC: 
0.6413542982814425


In [90]:
def bootstrapped_AUC(result):
    from sklearn.utils import resample
    from tqdm import tqdm

    n_iter = 10000
    roc_auc = list()
    prc_auc = list()


    for i in range(n_iter):
        result_sample = resample(result, n_samples = len(result),random_state=i)
        
        #Calculating AUROC for each sample
        y_ACTUAL= result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        fpr, tpr, thresholds = metrics.roc_curve(y_ACTUAL, scores_prob, pos_label=1)
        roc_auc.append(metrics.auc(fpr, tpr))

        #calculate AUPRC for each sample
        y_ACTUAL = result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        yhat = result_sample['pred_label']
        precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
        prc_auc.append(metrics.auc(recall,precision))
    
    return roc_auc, prc_auc

In [91]:
roc_auc_nn, pr_auc_nn = bootstrapped_AUC(results_nn)

In [92]:
dict = {'roc_auc_nn': roc_auc_nn,
        'pr_auc_nn': pr_auc_nn,
       }
pd.DataFrame(dict).describe(percentiles=[0.025,0.975])

,roc_auc_nn,pr_auc_nn
count,10000.000000,10000.000000
mean,0.916604,0.642171
std,0.020373,0.071938
min,0.814837,0.269615
2.5%,0.873992,0.495069
50%,0.917554,0.644365
97.5%,0.953322,0.777377
max,0.978876,0.885799
